In [1]:
#IMPORT LIBRARIES

In [2]:
import os; import pandas as pd; import numpy as np

In [3]:
#LOAD DATA

In [4]:
dataset = pd.read_csv('model_cm_60_r.csv')     #dataset with 60 variables
oot = pd.read_csv('out_of_sample_cm.csv')

backup = dataset
backup2 = oot

C:\Users\scienaptic\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (722,771,787,788,924,933,940,949,1156) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\scienaptic\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (986,1155) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
#START RUNNING CODE FROM HERE

dataset = backup
oot = backup2

In [31]:
#remove INF and NaNs, replace with special value

dataset = dataset.replace([np.inf, -np.inf], np.nan)
dataset = dataset.fillna(-999)
oot = oot.replace([np.inf, -np.inf], np.nan)
oot = oot.fillna(-999)

In [32]:
#select columns for data vs target
data = dataset.columns.intersection(oot.columns)
data = data.drop('target')  #remove target from training columns
data = data.drop('new_id')

In [33]:
#use dummy encoder to convert categorical variables to indicators

nonNumeric = data.drop(dataset[data].select_dtypes('number').columns)

dataNon = dataset[nonNumeric]
ootNon = oot[nonNumeric]

dataDummy = pd.get_dummies(dataNon)
ootDummy = pd.get_dummies(ootNon)

common = dataDummy.columns.intersection(ootDummy.columns) #to make sure they have the same columns
dataDummy = dataDummy[common] 
ootDummy = ootDummy[common]

In [34]:
#replace categorical variables with indicator variables 

dataset = dataset.drop(columns=nonNumeric)
oot = oot.drop(columns=nonNumeric)

dataset[dataDummy.columns] = dataDummy
oot[ootDummy.columns] = ootDummy

In [35]:
#update data then proceed to model

data = dataset.columns.intersection(oot.columns)
data = data.drop(['target', 'new_id'])

In [36]:
#   4-STEP MODELLING PROCESS: IMPORT WHICH MODEL, MAKE INSTANCE OF MODEL, TRAIN USING FIT, PREDICT LABELS OF TESTDATA

In [37]:
import statsmodels.api as sm

In [38]:
logit = sm.Logit(dataset['target'], dataset[data])

In [41]:
model = logit.fit_regularized()

C:\Users\scienaptic\Anaconda3\lib\site-packages\statsmodels\discrete\discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


MemoryError: 

In [ ]:
#assign data for testing

test = dataset

testData = test[data]
testTarget = test['target']

In [ ]:
prediction = xgb.predict_proba(testData)[:,1]     #predict probabilities, for ROC and KS

In [ ]:
#EVALUATE MODEL USING METRICS

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt


fpr, tpr, threshold = roc_curve(testTarget, prediction) 
roc_auc = auc(fpr,tpr)
roc_auc_score(testTarget, prediction)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show() 

In [ ]:
predictLabels = xgb.predict(testData)    #predict labels directly, for accuracy_score

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(testTarget, predictLabels)

In [ ]:
def rnkOrdering(data, bad_col, score_col, n_bkts):
    data['bucket'] = pd.qcut(data[score_col], n_bkts, duplicates='drop')
    grouped = data.groupby('bucket', as_index = True)
    agg1 = grouped[score_col].min()
    agg2 = grouped[score_col].max()
    agg3 = grouped[bad_col].count()
    agg4 = grouped[bad_col].sum()
    agg5 = np.round(grouped[score_col].mean()*100,2)
    
    ret_tbl = pd.DataFrame({"min_score":agg1, "max_score":agg2, "total":agg3, "Bads":agg4, "Mean_Score":agg5})
    ret_tbl["Goods"] = ret_tbl["total"] - ret_tbl["Bads"]
    ret_tbl["Bad_Rate"] = np.round(ret_tbl["Bads"]/ret_tbl["total"], 4)*100
    ret_tbl = ret_tbl.sort_index(axis=0, ascending=False)
    ret_tbl.index = range(n_bkts)
    
    ret_tbl["Cum_Bad"] = np.round((ret_tbl.Bads / ret_tbl.Bads.sum()).cumsum(), 4)*100
    ret_tbl["Cum_Good"] = np.round((ret_tbl.Goods / ret_tbl.Goods.sum()).cumsum(), 4)*100
    ret_tbl['KS'] = np.abs(ret_tbl["Cum_Bad"] - ret_tbl["Cum_Good"])
    
    gini = ret_tbl["Cum_Bad"][0]*ret_tbl["Cum_Good"][0]/(2*100*100)
    for i in range(1,n_bkts):
        gini += (ret_tbl["Cum_Bad"][i] + ret_tbl["Cum_Bad"][i-1])*(ret_tbl["Cum_Good"][i] - 
                                                                   ret_tbl["Cum_Good"][i-1])/(2*100*100)
    gini = np.round(2.*gini - 1., 4)*100
    
    ks = np.round(ret_tbl.KS.max(), 2)
    
    coltitles = ["min_score", "max_score", "Mean_Score", "total", "Goods", "Bads", "Bad_Rate", "Cum_Bad", "KS"]
    
    return ret_tbl[coltitles], {"Gini":gini, "KS":ks}

In [ ]:
testDF = testData
testDF['Bad'] = testTarget
testDF['Pred_Prob'] = prediction

print(rnkOrdering(testDF, "Bad", "Pred_Prob", 10))